In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets accelerate evaluate bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_id = "deepseek-ai/deepseek-coder-1.3b-base"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32022, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-06)
    (rotary_emb)

In [ ]:
import pandas as pd
from datasets import Dataset

df_train = pd.read_csv("/content/drive/MyDrive/nlp_project/train_hf_image_prompt_captioned_fullset.csv")
df_val = pd.read_csv("/content/drive/MyDrive/nlp_project/val_hf_image_prompt_captioned_fullset.csv")

hf_train = Dataset.from_pandas(df_train)
hf_val = Dataset.from_pandas(df_val)

In [ ]:
def tokenize_prompt_caption(example):
    if not example.get("prompt") or not example.get("caption"):
        return {"input_ids": [], "attention_mask": [], "labels": []}

    prompt = example['prompt']
    caption = example['caption']

    # Combine the full input
    full_text = f"### Prompt:\n{prompt}\n### Caption:\n{caption}"
    prompt_only = f"### Prompt:\n{prompt}\n### Caption:\n"

    # Tokenization
    full_enc = tokenizer(full_text, padding="max_length", truncation=True, max_length=256)
    prompt_enc = tokenizer(prompt_only, padding="max_length", truncation=True, max_length=256)

    # Use full_enc as input
    input_ids = full_enc["input_ids"]
    attention_mask = full_enc["attention_mask"]

    # Labels: to only predict the caption part
    labels = full_enc["input_ids"].copy()

    # Masking the prompt portion
    for i in range(len(prompt_enc["input_ids"])):
        if prompt_enc["input_ids"][i] != tokenizer.pad_token_id:
            labels[i] = -100

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
from torch.utils.data import DataLoader
hf_train_tok = hf_train.map(tokenize_prompt_caption, remove_columns=hf_train.column_names)
hf_val_tok = hf_val.map(tokenize_prompt_caption, remove_columns=hf_val.column_names)
hf_train_tok = hf_train.map(tokenize_prompt_caption, remove_columns=hf_train.column_names)
hf_val_tok = hf_val.map(tokenize_prompt_caption, remove_columns=hf_val.column_names)
hf_train_tok = hf_train_tok.filter(lambda x: len(x["input_ids"]) > 0 and len(x["labels"]) > 0)
hf_val_tok = hf_val_tok.filter(lambda x: len(x["input_ids"]) > 0 and len(x["labels"]) > 0)

def collate_fn(batch):
    batch = [x for x in batch if len(x["input_ids"]) > 0 and len(x["labels"]) > 0]

    if len(batch) == 0:
        return {
            "input_ids": torch.empty((0, 256), dtype=torch.long),
            "attention_mask": torch.empty((0, 256), dtype=torch.long),
            "labels": torch.empty((0, 256), dtype=torch.long),
        }

    input_ids = torch.tensor([x["input_ids"] for x in batch], dtype=torch.long)
    attention_mask = torch.tensor([x["attention_mask"] for x in batch], dtype=torch.long)
    labels = torch.tensor([x["labels"] for x in batch], dtype=torch.long)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

train_loader = DataLoader(hf_train_tok, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(hf_val_tok, batch_size=2, shuffle=False, collate_fn=collate_fn)

Map:   0%|          | 0/9470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

Map:   0%|          | 0/9470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9470 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1184 [00:00<?, ? examples/s]

In [ ]:
# Doing some checks to make sure we are on the right track
print("Prompt:", hf_train[0]['prompt'])
print("Caption:", hf_train[0]['caption'])

Prompt: a full body portrait of a beautiful holy roman nun of the valley meditative sacral pose catholic stages of the cross intricate elegant highly detailed digital painting artstation concept art smooth sharp focus illustration art by krenz cushart and artem demura and alphonse mucha
Caption: The image depicts a person wearing a flowing white robe with hands clasped in front, standing beside an ornate staff, all framed by a decorative background that enhances a mystical or sacred atmosphere.


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
model.train()
for epoch in range(3):
    print(f"\nEpoch {epoch+1}")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Skip if input is empty
        if input_ids.size(1) == 0 or input_ids.numel() == 0:
            print(f"Step {step}: Empty input — skipping.")
            continue

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            use_cache=False
        )

        loss = outputs.loss
        if torch.isnan(loss):
            print(f"Step {step}: Loss is NaN — skipping batch.")
            continue

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % 20 == 0:
            print(f"Step {step} | Loss: {loss.item():.4f}")


Epoch 1
Step 0 | Loss: 6.9710
Step 20 | Loss: 0.7607
Step 40 | Loss: 0.9257
Step 60 | Loss: 0.5559
Step 80 | Loss: 0.7008
Step 100 | Loss: 0.4460
Step 120 | Loss: 0.3746
Step 140 | Loss: 0.7877
Step 160 | Loss: 0.7869
Step 180 | Loss: 0.5813
Step 200 | Loss: 0.4511
Step 220 | Loss: 0.5739
Step 240 | Loss: 0.6412
Step 260 | Loss: 0.3515
Step 280 | Loss: 0.4403
Step 300 | Loss: 0.3275
Step 320 | Loss: 0.7425
Step 340 | Loss: 0.7286
Step 360 | Loss: 0.7146
Step 380 | Loss: 0.3672
Step 400 | Loss: 0.7762
Step 420 | Loss: 0.6797
Step 440 | Loss: 0.4197
Step 460 | Loss: 0.5252
Step 480 | Loss: 0.4682
Step 500 | Loss: 0.7540
Step 520 | Loss: 0.6175
Step 540 | Loss: 0.4160
Step 560 | Loss: 0.7782
Step 580 | Loss: 0.7015
Step 600 | Loss: 0.5390
Step 620 | Loss: 0.6817
Step 640 | Loss: 0.4437
Step 660 | Loss: 0.3231
Step 680 | Loss: 0.6265
Step 700 | Loss: 0.4945
Step 720 | Loss: 0.7154
Step 740 | Loss: 0.6203
Step 760 | Loss: 0.7361
Step 780 | Loss: 0.5477
Step 800 | Loss: 0.4667
Step 820 | Lo

In [ ]:
model.save_pretrained("/content/drive/My Drive/nlp_project/models")
tokenizer.save_pretrained("/content/drive/My Drive/nlp_project/models")

('/content/drive/My Drive/nlp_project/models/tokenizer_config.json',
 '/content/drive/My Drive/nlp_project/models/special_tokens_map.json',
 '/content/drive/My Drive/nlp_project/models/tokenizer.json')

In [ ]:
from tqdm import tqdm

model.eval()
generated_captions = []
reference_captions = []

with torch.no_grad():
    for batch in tqdm(val_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=64,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        clean_labels = []
        for label_seq in labels:
          clean_seq = [token_id.item() for token_id in label_seq if token_id != -100]
          clean_labels.append(clean_seq)

          refs = tokenizer.batch_decode(clean_labels, skip_special_tokens=True)
        generated_captions.extend([p.strip() for p in preds])
        reference_captions.extend([r.strip() for r in refs])


100%|██████████| 587/587 [17:33<00:00,  1.80s/it]


In [ ]:
print("Generated:", len(generated_captions), "| Reference:", len(reference_captions))
assert len(generated_captions) == len(reference_captions), "Mismatch in number of captions!"

#Get Bleu score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smooth = SmoothingFunction().method1
bleu_scores = [
    sentence_bleu([ref.split()], pred.split(), smoothing_function=smooth)
    for ref, pred in zip(reference_captions, generated_captions)
]

if bleu_scores:
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"Average BLEU Score: {avg_bleu:.4f}")

# BERTScore
from bert_score import score as bert_score

P, R, F1 = bert_score(generated_captions, reference_captions, lang="en", model_type="bert-base-uncased")
print(f"BERTScore F1: {F1.mean().item():.4f}")


Generated: 1173 | Reference: 1173
Average BLEU Score: 0.2172
BERTScore F1: 0.6798


In [ ]:
prompt = "A futuristic city skyline"

# Format like training examples
input_text = f"### Prompt:\n{prompt}\n### Caption:"
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=256
).to(model.device)

# Generate output
model.eval()
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

# Decode output
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

# Extract caption
if "### Caption:" in decoded:
    caption = decoded.split("### Caption:")[-1].strip()
else:
    caption = "[No caption generated]"

print(f"\nPrompt: {prompt}")
print(f"Generated Caption: {caption}")



Prompt: A futuristic city skyline
Generated Caption: The image depicts a sprawling, futuristic cityscape with vibrant pink and blue hues, showcasing vibrant pink and blue colors, showcasing vibrant street scenes in a tall, colorful prehistoric cityscape, showcasing tall buildings with vibrant yellow and surreal panorama. inspired
